In [1]:
import pyspark
from delta import *


In [2]:
builder=(pyspark.sql.SparkSession.builder.appName("Delta-table_External_Managed")
        .config("spark.sql.extensions","io.delta.sql.DeltaSparkSessionExtension")
        .config("spark.sql.catalog.spark_catalog","org.apache.spark.sql.delta.catalog.DeltaCatalog")
        )



In [3]:
spark=configure_spark_with_delta_pip(builder).getOrCreate()

:: loading settings :: url = jar:file:/Users/vk/Library/Python/3.9/lib/python/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/vk/.ivy2/cache
The jars for the packages stored in: /Users/vk/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-b00f764c-7289-4ac1-966b-5815de6ae343;1.0
	confs: [default]
	found io.delta#delta-core_2.12;2.2.0 in central
	found io.delta#delta-storage;2.2.0 in central
	found org.antlr#antlr4-runtime;4.8 in central
:: resolution report :: resolve 89ms :: artifacts dl 2ms
	:: modules in use:
	io.delta#delta-core_2.12;2.2.0 from central in [default]
	io.delta#delta-storage;2.2.0 from central in [default]
	org.antlr#antlr4-runtime;4.8 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   3   |   0   |   0   |   

23/04/08 20:33:47 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


# CREATE DELTA LAKE FILE WITH SAVE

In [30]:

columns=["Name","Country"]
data=[("QAZ","Singapore"),("WSX","India"),("EDC","United States")]
rdd=spark.sparkContext.parallelize(data)
df=rdd.toDF(columns)

In [46]:
#Save as delta file - Test purpose
df.write.format("delta").save("/tmp/my_first_delta_file")

In [47]:
%ls "/tmp/my_first_delta_file/"

_delta_log/
part-00000-ce90d84e-41e5-4841-a404-954c132d1f2d-c000.snappy.parquet
part-00002-b3f836ee-31a3-4fde-a7d1-71fafbc0bf32-c000.snappy.parquet
part-00005-6f4aa62b-deac-4d4d-a706-e2f12e2c63b5-c000.snappy.parquet
part-00007-04ee1879-8034-4fad-81a8-f7c5bb829f44-c000.snappy.parquet


In [48]:
%cat "/tmp/my_first_delta_file/_delta_log/00000000000000000000.json"

{"commitInfo":{"timestamp":1680962086347,"operation":"WRITE","operationParameters":{"mode":"ErrorIfExists","partitionBy":"[]"},"isolationLevel":"Serializable","isBlindAppend":true,"operationMetrics":{"numFiles":"4","numOutputRows":"3","numOutputBytes":"2614"},"engineInfo":"Apache-Spark/3.3.2 Delta-Lake/2.2.0","txnId":"4f832f7b-bb53-445e-a810-bbb3a1c6d7f7"}}
{"protocol":{"minReaderVersion":1,"minWriterVersion":2}}
{"metaData":{"id":"3cb3bc92-1245-4641-8bcf-68cb9d6b6314","format":{"provider":"parquet","options":{}},"schemaString":"{\"type\":\"struct\",\"fields\":[{\"name\":\"Name\",\"type\":\"string\",\"nullable\":true,\"metadata\":{}},{\"name\":\"Country\",\"type\":\"string\",\"nullable\":true,\"metadata\":{}}]}","partitionColumns":[],"configuration":{},"createdTime":1680962086083}}
{"add":{"path":"part-00002-b3f836ee-31a3-4fde-a7d1-71fafbc0bf32-c000.snappy.parquet","partitionValues":{},"size":739,"modificationTime":1680962086344,"dataChange":true,"stats":"{\"numRecords\":1,\"minValues\

In [51]:
spark.read.format("delta").load("/tmp/my_first_delta_file/").show()

+----+-------------+
|Name|      Country|
+----+-------------+
| EDC|United States|
| QAZ|    Singapore|
| WSX|        India|
+----+-------------+



In [52]:
df.createOrReplaceTempView("Delta_table_External_Managed")

In [54]:
spark.sql("DESCRIBE DETAIL Delta_table_External_Managed").show(truncate=False)

+------+----+------------------------------------+-----------+-------------------------------------------------------------------------------+-----------------------+------------+----------------+--------+-----------+----------+----------------+----------------+
|format|id  |name                                |description|location                                                                       |createdAt              |lastModified|partitionColumns|numFiles|sizeInBytes|properties|minReaderVersion|minWriterVersion|
+------+----+------------------------------------+-----------+-------------------------------------------------------------------------------+-----------------------+------------+----------------+--------+-----------+----------+----------------+----------------+
|delta |null|default.delta_table_external_managed|           |file:/Users/vk/Documents/Deltalake/spark-warehouse/delta_table_external_managed|2023-04-08 21:41:33.194|null        |[]              |null    |null  

In [56]:
spark.sql("DESCRIBE TABLE EXTENDED Delta_table_External_Managed").show()

+--------+---------+-------+
|col_name|data_type|comment|
+--------+---------+-------+
|    Name|   string|   null|
| Country|   string|   null|
+--------+---------+-------+



# CREATE EXTERNAL DELTA LAKE TABLE

In [57]:
df.write.format('delta').option("path","/tmp/external_table").saveAsTable("default.my_external_table")

In [58]:
spark.sql("DESCRIBE DETAIL my_external_table").show(truncate=False)

+------+------------------------------------+-------------------------+-----------+------------------------+-----------------------+-----------------------+----------------+--------+-----------+----------+----------------+----------------+
|format|id                                  |name                     |description|location                |createdAt              |lastModified           |partitionColumns|numFiles|sizeInBytes|properties|minReaderVersion|minWriterVersion|
+------+------------------------------------+-------------------------+-----------+------------------------+-----------------------+-----------------------+----------------+--------+-----------+----------+----------------+----------------+
|delta |9b83b2f4-4849-43a7-94cb-deaea4e8d953|default.my_external_table|null       |file:/tmp/external_table|2023-04-08 22:13:48.159|2023-04-08 22:13:48.424|[]              |3       |2217       |{}        |1               |2               |
+------+--------------------------------

In [59]:
spark.sql("select * from default.my_external_table").show()

+----+-------------+
|Name|      Country|
+----+-------------+
| EDC|United States|
| QAZ|    Singapore|
| WSX|        India|
+----+-------------+



In [60]:
spark.sql("DESCRIBE TABLE EXTENDED my_external_table").show(truncate=False)

+----------------------------+---------------------------------------------------+-------+
|col_name                    |data_type                                          |comment|
+----------------------------+---------------------------------------------------+-------+
|Name                        |string                                             |       |
|Country                     |string                                             |       |
|                            |                                                   |       |
|# Partitioning              |                                                   |       |
|Not partitioned             |                                                   |       |
|                            |                                                   |       |
|# Detailed Table Information|                                                   |       |
|Name                        |default.my_external_table                          |       |

# CREATE DELTA LAKE MANAGED TABLE

In [61]:
df.write.format("delta").mode("overwrite").saveAsTable("delta_lake_managed_table")

In [63]:
spark.table(tableName="delta_lake_managed_table").show(truncate=False)

+----+-------------+
|Name|Country      |
+----+-------------+
|EDC |United States|
|QAZ |Singapore    |
|WSX |India        |
+----+-------------+



In [64]:
%ls "spark-warehouse/delta_lake_managed_table/"


_delta_log/
part-00000-f34aa72e-6a89-4529-ad59-dc77903b7960-c000.snappy.parquet
part-00002-e428506b-ae46-42ea-9bd0-9874dbe4bc0b-c000.snappy.parquet
part-00005-1a1c49ee-28bb-4472-9c73-3570ad97408e-c000.snappy.parquet
part-00007-f828fd8a-2976-4a76-87ad-6fc03dac803f-c000.snappy.parquet


In [65]:
spark.sql("DESCRIBE TABLE EXTENDED delta_lake_managed_table").show(truncate=False)

+----------------------------+---------------------------------------------------------------------------+-------+
|col_name                    |data_type                                                                  |comment|
+----------------------------+---------------------------------------------------------------------------+-------+
|Name                        |string                                                                     |       |
|Country                     |string                                                                     |       |
|                            |                                                                           |       |
|# Partitioning              |                                                                           |       |
|Not partitioned             |                                                                           |       |
|                            |                                                  